In [ ]:
import torch
torch.__version__

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from torch.autograd import Variable


## PyTorch로 첫 번째 신경망 만들기

학습 데이터를 목적 변수(출력 배열)에 매핑하는 것을 학습하는 신경망을 만들어 볼 것입니다. 우리는 주문형 비디오 서비스를 제공하는 Wondemovies라는 큰 온라인 회사에서 일한다고 가정하겠습니다. 플랫폼에서 영화를 보는 사용자가 보낸 평균 시간을 피처로하는 학습 데이터셋을 갖고 있습니다. 이 학습 데이터셋으로 다음 주에 사용자가 플랫폼에 얼마나 많은 시간을 소비할 것인지 예측하고 싶습니다. 이것은 상상의 유스 케이스입니다. 너무 많이 고민할 필요는 없습니다. 이러한 솔루션을 구축하기 위해서 다음과 같은 작업을 수행합니다. 

1. 데이터 준비 : **get_data()** 함수는 입력과 출력 데이터로 구성된 Tensor 배열을 제공합니다. 
2. 학습 파라미터 생성 : **get_weights()** 함수는 임의의 수를 포함하는 Tensor를 제공합니다. 이 임의의 수는 이 문제의 해법을 구하기 위해서 최적화 됩니다. 
3. 네트워크 모델 : **simple_network()** 는 선형 규칙을 적용하고 가중치에 입력 데이터를 곱하고 바이어스 항 (y = Wx + b)을 더하여 입력 데이터에 대한 출력을 생성합니다
4. 오차 : **loss_fn()** 함수는 모델의 얼마나 좋은지에 대한 정보를 제공합니다. 
5. 옵티마이저 : **optimize()** 함수는 초기에 생성 된 임의의 가중치를 조정하여 모델이 목표 값을보다 정확하게 계산하도록합니다.

In [ ]:
# 학습 데이터
def get_data():
    train_X = np.asarray([3.3,4.4,5.5,6.71,6.93,4.168,9.779,6.182,7.59,2.167,
                         7.042,10.791,5.313,7.997,5.654,9.27,3.1])
    train_Y = np.asarray([1.7,2.76,2.09,3.19,1.694,1.573,3.366,2.596,2.53,1.221,
                         2.827,3.465,1.65,2.904,2.42,2.94,1.3])
    dtype = torch.FloatTensor
    X = Variable(torch.from_numpy(train_X).type(dtype),requires_grad=False).view(17,1)
    y = Variable(torch.from_numpy(train_Y).type(dtype),requires_grad=False)
    return X,y

def plot_variable(x,y,z='',**kwargs):
    l = []
    for a in [x,y]:
        #1.0 업그레이드에서 필요 없는 코드
        #if type(a) == Variable:
        l.append(a.data.numpy())
    plt.plot(l[0],l[1],z,**kwargs)

def get_weights():
    w = Variable(torch.randn(1),requires_grad = True)
    b = Variable(torch.randn(1),requires_grad=True)
    return w,b

def simple_network(x):
    y_pred = torch.matmul(x,w)+b
    return y_pred

def loss_fn(y,y_pred):
    loss = (y_pred-y).pow(2).sum()
    for param in [w,b]:
        if not param.grad is None: param.grad.data.zero_()
    loss.backward()
    return loss.data


def optimize(learning_rate):
    w.data -= learning_rate * w.grad.data
    b.data -= learning_rate * b.grad.data

learning_rate = 1e-4

In [ ]:
x,y = get_data()               # x - 학습 데이터, y - 목적 변수(Target Variables)
w,b = get_weights()           # w,b - 학습 파라미터
for i in range(500):
    y_pred = simple_network(x) # wx + b를 계산하는 함수
    loss = loss_fn(y,y_pred)   # y와 y_pred의 차의 제곱 합을 계산
    if i % 50 == 0: 
        print(loss)
    optimize(learning_rate)    # 오차를 최소화하도록 w, b를 조정

In [ ]:
x_numpy = x.data.numpy()
plot_variable(x,y,'ro')
plot_variable(x,y_pred,label='Fitted line')

## Tensor 소개

### Scalar

In [ ]:
x = torch.rand(10)
x.size()

### Vector

In [ ]:
temp = torch.FloatTensor([23,24,24.5,26,27.2,23.0])
temp.size()

### Matrix

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()
print(boston.data.shape)

In [ ]:
boston.feature_names

In [ ]:
boston_tensor = torch.from_numpy(boston.data)
boston_tensor.size()

In [ ]:
boston_tensor[:2]

In [ ]:
boston_tensor[:10,:5]

### 3d- tensor

In [ ]:
from PIL import Image

panda = np.array(Image.open('../images/B09301_03_12.jpg').resize((224,224)))
panda_tensor = torch.from_numpy(panda)
panda_tensor.size()

In [ ]:
plt.imshow(panda);

### Tensor 자르기

In [ ]:
sales = torch.FloatTensor([1000.0,323.2,333.4,444.5,1000.0,323.2,333.4,444.5])

In [ ]:
sales[:5]

In [ ]:
sales[:-5]

In [ ]:
plt.imshow(panda_tensor[:,:,0].numpy());

In [ ]:
plt.imshow(panda_tensor[25:175,60:130,0].numpy());

### Tensor에서 특정 요소 가져오기 

In [ ]:
#torch.eye(shape) produces an diagonal matrix with 1 as it diagonal #elements.
sales = torch.eye(3,3)
sales[0,1]

### 4D Tensor

In [ ]:
from glob import glob
#디스크에서 고양이 이미지 읽기
data_path='/Users/vishnu/Documents/fastAIPytorch/fastai/courses/dl1/data/dogscats/train/cats/'
cats = glob(data_path+'*.jpg')
#Convert images into numpy arrays
cat_imgs = np.array([np.array(Image.open(cat).resize((224,224))) for cat in
cats[:64]])
cat_imgs = cat_imgs.reshape(-1,224,224,3)
cat_tensors = torch.from_numpy(cat_imgs)
cat_tensors.size()

### Tensor 덧셈 연산과 곱셈 연산

In [ ]:
#Various ways you can perform tensor addition
a = torch.rand(2,2) 
b = torch.rand(2,2)
c = a + b
d = torch.add(a,b)
#덧셈 연산
a.add_(5)

#두 텐서의 곱셈

a*b
a.mul(b)
#in-place 곱셈
a.mul_(b)

### GPU 지원

In [ ]:
%%time
a = torch.rand(10000,10000)
b = torch.rand(10000,10000)

a.matmul(b)

- log on Mac
```
CPU times: user 1min 8s, sys: 2.44 s, total: 1min 11s
Wall time: 20.7 s    
```    

In [ ]:
#GPU에 Tensor 이동
from torch import cuda 

if cuda.is_available(): 
    a = a.cuda()
    b = b.cuda()
    a.matmul(b)
#Time taken : 11.2 µs  

### Variables

In [ ]:
from torch.autograd import Variable
x = Variable(torch.ones(2,2),requires_grad=True)
y = x.mean()
y.backward()
x.grad

In [ ]:
x.grad_fn

In [ ]:
x.data

In [ ]:
y.grad_fn

### 신경망에 사용할 데이터 생성

In [ ]:
def get_data():
    train_X = np.asarray([3.3,4.4,5.5,6.71,6.93,4.168,9.779,6.182,7.59,2.167,
                         7.042,10.791,5.313,7.997,5.654,9.27,3.1])
    train_Y = np.asarray([1.7,2.76,2.09,3.19,1.694,1.573,3.366,2.596,2.53,1.221,
                         2.827,3.465,1.65,2.904,2.42,2.94,1.3])
    dtype = torch.FloatTensor
    X = Variable(torch.from_numpy(train_X).type(dtype),requires_grad=False).view(17,1)
    y = Variable(torch.from_numpy(train_Y).type(dtype),requires_grad=False)
    return X,y

### 학습 파라미터 생성

In [ ]:
def get_weights():
    w = Variable(torch.randn(1),requires_grad = True)
    b = Variable(torch.randn(1),requires_grad=True)
    return w,b

### 신경망 구현

In [ ]:
def simple_network(x):
    y_pred = torch.matmul(x,w)+b
    return y_pred

### 파이토치 신경망 구현

In [ ]:
import torch.nn as nn
f = nn.Linear(17,1) # Much simpler.
f

### 오차 함수 구현

In [ ]:
def loss_fn(y,y_pred):
    loss = (y_pred-y).pow(2).sum()
    for param in [w,b]:
        if not param.grad is None: param.grad.data.zero_()
    loss.backward()
    return loss.data[0]

### 옵티마이저 구현

In [ ]:
def optimize(learning_rate):
    w.data -= learning_rate * w.grad.data
    b.data -= learning_rate * b.grad.data

## 데이터 로딩

### Dataset 정의

In [ ]:
from torch.utils.data import Dataset
class DogsAndCatsDataset(Dataset):
    def __init__(self,):
        pass
    def __len__(self):
        pass
    def __getitem__(self,idx):
        pass

In [ ]:
class DogsAndCatsDataset(Dataset):
    def __init__(self,root_dir,size=(224,224)):
        self.files = glob(root_dir)
        self.size = size
    def __len__(self):
        return len(self.files)
    def __getitem__(self,idx):
        img = np.asarray(Image.open(self.files[idx]).resize(self.size))
        label = self.files[idx].split('/')[-2]
        return img,label

### DogsAndCatsDataset을 순회하는 DataLoader 정의

In [ ]:
from torch.utils.data import Dataset, DataLoader

#dataloader = DataLoader(DogsAndCatsDataset,batch_size=32,num_workers=2)
#for imgs , labels in dataloader:
        #학습 데이터셋에 딥러닝 네트워크 적용
#    pass